## The Kaggle Rossman Cometition

This was a kaggle competition to forecast sales at a pharmacy chain/dept store in Europe. It was run back in 2015.

The aim of this notebook is

(a) to see how to use sklearn pipelines
(b) to understand some aspects of feature engineering that come in with continuous and categorical variables
(c) to capture results from validation
(d) to investigate the use of categorical "embeddings" to improve performance of a multi-layer percepton (in another version)

### Preprocessing

In [1]:
import numpy as np
import scipy.stats
import scipy.special

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
%matplotlib inline

In [2]:
from pathlib import Path

In [3]:
data = Path('./data')

We engage in some cleaning. A lot of cleaning of this dataset has already been done for us. Some features have been created. In particular we moved from dates to week-of-year, day-of week, etc. For example the 49th and 50th weeks of the year may have higher sales!

In [4]:
train_df = pd.read_csv(data/"train_clean.csv.gz", compression='gzip').drop(['index', 'PromoInterval'], axis=1)
test_df = pd.read_csv(data/"test_clean.csv.gz", compression='gzip').drop(['index', 'PromoInterval'], axis=1)

In [5]:
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,1,1,4,2015-09-17,1.0,1,False,0,2015,9,...,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2,3,4,2015-09-17,1.0,1,False,0,2015,9,...,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
2,3,7,4,2015-09-17,1.0,1,False,0,2015,9,...,115,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
3,4,8,4,2015-09-17,1.0,1,False,0,2015,9,...,115,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
4,5,9,4,2015-09-17,1.0,1,False,0,2015,9,...,105,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0


In [6]:
test_df.columns

Index(['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'State', 'file', 'week', 'trend',
       'file_DE', 'week_DE', 'trend_DE', 'Date_DE', 'State_DE', 'Month_DE',
       'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE', 'Is_month_end_DE',
       'Is_month_start_DE', 'Is_quarter_end_DE', 'Is_quarter_start_DE',
       'Is_year_end_DE', 'Is_year_start_DE', 'Elapsed_DE', 'Max_TemperatureC',
       'Mean_TemperatureC', 'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC',
       'Min_DewpointC', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_Pre

In [7]:
train_df['Events'] = train_df['Events'].fillna('None')
test_df['Events'] = test_df['Events'].fillna('None')

And in log-transforming the dependent variable because it is long-tailed. Histo it to see this

In [44]:
# your code here
plt.hist(train_df['Sales']);

KeyError: 'Sales'

In [8]:
train_resp = np.log(train_df['Sales'].copy())
train_df = train_df.drop('Sales', axis=1)

Lets get some idea about our dataset.

In [9]:
train_df.head()

,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,1,5,2015-07-31,555,1,1,False,1,2015,7,...,57,0,0,0,5.0,0.0,5.0,7.0,0.0,5.0
1,2,5,2015-07-31,625,1,1,False,1,2015,7,...,67,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0
2,3,5,2015-07-31,821,1,1,False,1,2015,7,...,57,0,0,0,5.0,0.0,5.0,5.0,0.0,5.0
3,4,5,2015-07-31,1498,1,1,False,1,2015,7,...,67,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0
4,5,5,2015-07-31,559,1,1,False,1,2015,7,...,57,0,0,0,5.0,0.0,5.0,1.0,0.0,1.0


In [10]:
train_df.shape, test_df.shape

((844338, 90), (41088, 90))

In [11]:
train_df.Date # latest date first

0         2015-07-31
1         2015-07-31
2         2015-07-31
3         2015-07-31
4         2015-07-31
5         2015-07-31
6         2015-07-31
7         2015-07-31
8         2015-07-31
9         2015-07-31
10        2015-07-31
11        2015-07-31
12        2015-07-31
13        2015-07-31
14        2015-07-31
15        2015-07-31
16        2015-07-31
17        2015-07-31
18        2015-07-31
19        2015-07-31
20        2015-07-31
21        2015-07-31
22        2015-07-31
23        2015-07-31
24        2015-07-31
25        2015-07-31
26        2015-07-31
27        2015-07-31
28        2015-07-31
29        2015-07-31
             ...    
844308    2013-01-02
844309    2013-01-02
844310    2013-01-02
844311    2013-01-02
844312    2013-01-02
844313    2013-01-02
844314    2013-01-02
844315    2013-01-02
844316    2013-01-02
844317    2013-01-02
844318    2013-01-02
844319    2013-01-02
844320    2013-01-02
844321    2013-01-01
844322    2013-01-01
844323    2013-01-01
844324    201

### Types of variables and cardinality

We make a note of which variables are categorical and which are not. This is a choice. If cardinality is not too high, binning or categorizing can be beneficial. Often this will be true for integer valued variables.

In [12]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw', 'Promo', 'SchoolHoliday']

cont_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday']

We look for missing data and store the column names where this happend in the continuous data

In [13]:
nacols=[]
for v in cont_vars:
    if np.sum(train_df[v].isnull()) > 0:
        nacols.append(v)
        print(v, np.sum(train_df[v].isnull()))

CompetitionDistance 2186
CloudCover 68056


And look at some cardinalities: since we have none below 10, we dont engage in binning.

In [14]:
# your code here
for k in cont_vars:
    print(k, train_df[k].unique().shape[0])
    if train_df[k].unique().shape[0] < 10:
        print(train_df[k].unique())

CompetitionDistance 655
Max_TemperatureC 50
Mean_TemperatureC 45
Min_TemperatureC 40
Max_Humidity 50
Mean_Humidity 71
Min_Humidity 93
Max_Wind_SpeedKm_h 42
Mean_Wind_SpeedKm_h 27
CloudCover 10
trend 67
trend_DE 38
AfterStateHoliday 136
BeforeStateHoliday 147


We do a similar looksie on the categorical variables. Some of these have many levels. Is there really that much information in 1115 store labels. Can we get some compression to increase our signal-to-noise?

In [15]:
for k in cat_vars:
    print(k, train_df[k].unique().shape[0])
    if train_df[k].unique().shape[0] < 50:
        print(train_df[k].unique())

Store 1115
DayOfWeek 7
[5 4 3 2 1 7 6]
Year 3
[2015 2014 2013]
Month 12
[ 7  6  5  4  3  2  1 12 11 10  9  8]
Day 31
[31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8
  7  6  5  4  3  2  1]
StateHoliday 2
[False  True]
CompetitionMonthsOpen 25
[24  3 19  9  0 16 17  7 15 22 11 13  2 23 12  4 10  1 14 20  8 18  6 21
  5]
Promo2Weeks 26
[ 0 25 17  8 13 24 16  7 12 23 15  6 11 22 14  5 10 21  4  9 20  3 19  2
 18  1]
StoreType 4
['c' 'a' 'd' 'b']
Assortment 3
['a' 'c' 'b']
CompetitionOpenSinceYear 23
[2008 2007 2006 2009 2015 2013 2014 2000 2011 1900 2010 2005 1999 2003
 2012 2004 2002 1961 1995 2001 1990 1994 1998]
Promo2SinceYear 8
[1900 2010 2011 2012 2009 2014 2015 2013]
State 12
['HE' 'TH' 'NW' 'BE' 'SN' 'SH' 'HB,NI' 'BY' 'BW' 'RP' 'ST' 'HH']
Week 52
Events 22
['Fog' 'None' 'Rain' 'Rain-Thunderstorm' 'Fog-Rain'
 'Rain-Hail-Thunderstorm' 'Fog-Rain-Thunderstorm' 'Thunderstorm'
 'Rain-Hail' 'Fog-Thunderstorm' 'Rain-Snow' 'Fog-Rain-Hail-Thunderstorm'
 'Snow' 'Rain-

### Creating a validation set

The construction of a validation or "development" set is not always a `test_train_split` deal. Here we create a validation set of "latest" data, cireesponding oin date and size to what we have in the test set. Hopefully this will make sure we have similar distributions of features and outcomes on both.

In [16]:
cut = train_df['Date'][(train_df['Date'] == train_df['Date'][len(test_df)])].index.max()
cut

41395

In [17]:
valid_idx = range(cut)
train_idx = list(np.setdiff1d(range(train_df.shape[0]), valid_idx))

In [18]:
trdf = train_df.iloc[train_idx]
vadf = train_df.iloc[valid_idx]

In [19]:
trdf.shape, vadf.shape

((802943, 90), (41395, 90))

### Transformation Pipelines

Ok, now we'll use the new `ColumnTransformer`, with imputation, missing-data indicators, the new `OrdinalEncoder`, and the usual Standard Scaling.

In [20]:
from sklearn.impute import SimpleImputer,MissingIndicator
from sklearn.pipeline import make_pipeline, make_union, Pipeline

In [21]:
impu = SimpleImputer(strategy="median") # create a median imputer

We do the missing indicator separately as it creates a new column. It is possible to do this in the pipeline flow in `sklearn` using a union, but subsequent scaling wants to scale this indicator since the categorical list does not include the new columns.

In [22]:
mi = MissingIndicator() # create, fit, and transform a missingness indicator
mi.fit(trdf[nacols])
Xtrmi = mi.transform(trdf[nacols])
Xvami = mi.transform(vadf[nacols])

In [23]:
Xtrmi[4460,:]

array([False,  True])

In [24]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
ss = StandardScaler()
oe = OrdinalEncoder()

In [25]:
trdf_cat = trdf[cat_vars]
trdf_cont = trdf[cont_vars]

We construct two pipelines, one for categoricals and one for continuous variables

In [26]:
cont_pipe = Pipeline([("imp",impu), ("scale", ss)])


In [27]:
cat_pipe = Pipeline([("categorify", oe)])

And combine them here in a transformer list.

In [28]:
transformers = [('cat', cat_pipe, cat_vars),
                    ('cont', cont_pipe, cont_vars)]

Now we use a `ColumnTransformer` to combine these.

In [29]:
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=transformers)

In [30]:
ct.fit(trdf)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('cat', Pipeline(memory=None,
     steps=[('categorify', OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>))]), ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 'StoreType', 'Assortment', 'CompetitionOpenSinceYear', 'P...ean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE', 'AfterStateHoliday', 'BeforeStateHoliday'])])

In [31]:
Xtr = ct.transform(trdf)
Xval = ct.transform(vadf)

In [32]:
Xtr.shape, Xtrmi.shape

((802943, 37), (802943, 2))

We concatenate the old indicators back in. The transformer lists all the categoricals first, since thats the first item in `transformers`, so we pre-pend.

In [33]:
Xtrain = np.concatenate([Xtrmi, Xtr], axis=1)
Xtrain.shape

(802943, 39)

In [34]:
Xvalid = np.concatenate([Xvami, Xval], axis=1)
Xvalid.shape

(41395, 39)

sklearn-pipelines lose our nice pandas names. so we bring them back.

In [35]:
cols = trdf.columns
actcols = []
actcolcount = 0
nacols_cat = []
for k in nacols:
    actcols.append((k+'_missing', 'cont'))
    nacols_cat.append(k+'_missing')
    actcolcount+=1
for k in cat_vars+cont_vars:
    if k in cat_vars:
        actcols.append((k, "cat"))
        actcolcount+=1
    if k in cont_vars:
        actcols.append((k, "cont"))
        actcolcount+=1
        
list(enumerate(actcols)), actcolcount

([(0, ('CompetitionDistance_missing', 'cont')),
  (1, ('CloudCover_missing', 'cont')),
  (2, ('Store', 'cat')),
  (3, ('DayOfWeek', 'cat')),
  (4, ('Year', 'cat')),
  (5, ('Month', 'cat')),
  (6, ('Day', 'cat')),
  (7, ('StateHoliday', 'cat')),
  (8, ('CompetitionMonthsOpen', 'cat')),
  (9, ('Promo2Weeks', 'cat')),
  (10, ('StoreType', 'cat')),
  (11, ('Assortment', 'cat')),
  (12, ('CompetitionOpenSinceYear', 'cat')),
  (13, ('Promo2SinceYear', 'cat')),
  (14, ('State', 'cat')),
  (15, ('Week', 'cat')),
  (16, ('Events', 'cat')),
  (17, ('Promo_fw', 'cat')),
  (18, ('Promo_bw', 'cat')),
  (19, ('StateHoliday_fw', 'cat')),
  (20, ('StateHoliday_bw', 'cat')),
  (21, ('SchoolHoliday_fw', 'cat')),
  (22, ('SchoolHoliday_bw', 'cat')),
  (23, ('Promo', 'cat')),
  (24, ('SchoolHoliday', 'cat')),
  (25, ('CompetitionDistance', 'cont')),
  (26, ('Max_TemperatureC', 'cont')),
  (27, ('Mean_TemperatureC', 'cont')),
  (28, ('Min_TemperatureC', 'cont')),
  (29, ('Max_Humidity', 'cont')),
  (30, ('

### Time to learn

We first split the y (the log of the y, really)

In [36]:
ytrain = train_resp[train_idx]
yvalid = train_resp[list(valid_idx)]
ytrain.shape, yvalid.shape

((802943,), (41395,))

In [41]:
Xtrain.shape, Xvalid.shape

((802943, 39), (41395, 39))

In [48]:
import pickle
import gzip
to_save = dict(key=list(enumerate(actcols)), Xtrain=Xtrain, Xvalid=Xvalid, ytrain=ytrain, yvalid=yvalid)
with gzip.open("data/processed.pkl.gz", "wb") as fd:
    pickle.dump(to_save, fd)

In [49]:
with gzip.open("data/processed.pkl.gz", "rb") as fd:
    reget = pickle.load(fd)

In [50]:
reget.keys()

dict_keys(['key', 'Xtrain', 'Xvalid', 'ytrain', 'yvalid'])

## Baseline Logistic Regression

In [45]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import ParameterGrid
param_grid = {'alpha': [1e-6, 1e-5, 1e-3, 0.001, 0.01, 0.1, 1, 10, 100]}

## Obtain the minimum

In [47]:
from sklearn.metrics import mean_squared_error

# your code here
for p in ParameterGrid(param_grid):
    est = Ridge()
    est.set_params(**p)
    est.fit(Xtrain, ytrain)
    ypred = est.predict(Xvalid)
    ypredtrain = est.predict(Xtrain)
    mse = mean_squared_error(ypred, yvalid)
    msetr = mean_squared_error(ypredtrain, ytrain)
    print("C", p, "MSE Valid", mse, "Train", msetr, 'VScore', est.score(Xvalid, yvalid))

C {'alpha': 1e-06} MSE Valid 0.12686369496129776 Train 0.13746464595659655 VScore 0.260601355295936
C {'alpha': 1e-05} MSE Valid 0.12686369496178454 Train 0.1374646459565966 VScore 0.2606013552930989
C {'alpha': 0.001} MSE Valid 0.12686369501533346 Train 0.13746464595659655 VScore 0.26060135498100034
C {'alpha': 0.001} MSE Valid 0.12686369501533346 Train 0.13746464595659655 VScore 0.26060135498100034
C {'alpha': 0.01} MSE Valid 0.12686369550214394 Train 0.13746464595660104 VScore 0.2606013521437267
C {'alpha': 0.1} MSE Valid 0.1268637003705061 Train 0.13746464595704072 VScore 0.26060132376949097
C {'alpha': 1} MSE Valid 0.12686374907981293 Train 0.137464646000974 VScore 0.260601039877433
C {'alpha': 10} MSE Valid 0.1268642387087756 Train 0.1374646503608408 VScore 0.26059818617691977
C {'alpha': 100} MSE Valid 0.12686935825463155 Train 0.13746505506237522 VScore 0.26056834796931794
